In [1]:
def HTC_Y(self, nodeA):
        parents = nodeA.parents
        siblings = nodeA.siblings
        half_treks = []
        Y = []
        HTC = False
        
        if len(parents) == 0:
            nodeA.set_Yv(Y)
            HTC = True

        for parent in parents:
            parent_treks = []
            
            for variable in self.variables:
                if variable != nodeA and not variable in siblings:
                    var_to_parent = self.get_half_treks(variable, parent)
                    
                    for trek in var_to_parent:
                        parent_treks.append(trek)
            half_treks.append(parent_treks)
        
        combinations = list(itertools.product(*half_treks))

        for comb in combinations:
            if self.sided_interaction(comb) == False and HTC == False:
                for item in comb:
                    Y.append(item[0][0])
                nodeA.set_Yv(Y)
                HTC = True
                
        return nodeA.get_Yv()

In [2]:
def sided_interaction(self, tracks):
    left_side = []
    right_side = []
    
    for track in tracks:
        track_left = [track[0][0]]
        track_right = [track[-1][0]]
        prev_step = track[0]

        for step in track:
            if step[1] == "l" and not prev_step[0] in track_left:
                track_left.append(prev_step[0])
            elif step[1] == "r" and not prev_step[0] in track_right:
                track_right.append(prev_step[0])
            elif step[1] == "b" and not prev_step[0] in track_left:
                track_left.append(prev_step[0])
            
            prev_step = step
            
        left_side += track_left
        right_side += track_right

    unique_left = list(set(left_side))
    unique_right = list(set(right_side))

    if len(unique_left) == len(left_side) and len(unique_right) == len(right_side):
        sided_interaction = False
    else:
        sided_interaction = True

    return sided_interaction

In [3]:
def estimate_Lambda(self, Sigma):
    if self.HTC_order == None:
        self.is_HTC_identifiable, self.HTC_order = HTC_identifiable(self)
    
    if self.is_HTC_identifiable == False:
        raise Exception("Graph not HTC idenifiable")

    Lambda_est = self.GF.Zeros((self.N_nodes, self.N_nodes))

    for var in self.HTC_order:
        A = self.GF.Zeros((len(var.Yv), len(var.Yv)))
        b = self.GF.Zeros(len(var.Yv))
        
        for row_id, y in enumerate(var.Yv):
            for col_id, parent in enumerate(var.parents):
                y_id = self.var_to_int[y]
                parent_id = self.var_to_int[parent]
                var_id = self.var_to_int[var]

                I_min_lamb_T = (self.GF.Identity(self.N_nodes) - Lambda_est).T
                
                if y in htr(self, var):
                    A[row_id][col_id] = np.matmul(I_min_lamb_T[y_id, :], Sigma[:, parent_id])
                    b[row_id] = np.matmul(I_min_lamb_T[y_id, :], Sigma[:, var_id])
                else:
                    A[row_id][col_id] = Sigma[y_id][parent_id]
                    b[row_id] = Sigma[y_id][var_id]

        Lambda_parents = np.matmul(np.linalg.inv(A), b)
        
        for indx, parent in enumerate(var.parents):
            parent_id = self.var_to_int[parent]
            Lambda_est[parent_id][var_id] = Lambda_parents[indx]

    return Lambda_est

In [4]:
def estimate_Omega(self, Sigma):
        Lambda_est = self.estimate_Lambda(Sigma)
        I_min_lamb = (self.GF.Identity(self.N_nodes) - Lambda_est)
        Omega_est = np.matmul(I_min_lamb.T, np.matmul(Sigma, I_min_lamb))

        return Omega_est